# Introduction

In July 2002 Bo Pang, Lillian Lee, and Shivakumar Vaithyanathan published [*Thumbs up? Sentiment Classification using Machine Learning Techniques.*](https://aclanthology.org/W02-1011/) at EMNLP, one of the earliest works of using machine learning for Sentiment Classification.
It was an influential paper, [winning a test of time award](https://vimeo.com/280330773) at NAACL 2018, and at the time of writing has over 11,000 citations.
This work led to their follow up [Seeing stars: Exploiting class relationships for sentiment categorization with respect to rating scales](https://arxiv.org/abs/cs/0506075) and this dataset was the basis for the Stanford Sentiment Treebank dataset released in [Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank](https://aclanthology.org/D13-1170.pdf) by Socher et al., which is widely used partly because of it's inclusion in GLUE.

This paper aims to show that classifying the sentiment of movie reviews is a more challenging problem to develop machine learning techniques on than the existing topic classification problems, and motivate further work (in which they were successful!)
They do this by building a self-labelled dataset of polar movie reviews from Usenet and then show baseline classifiers don't work as well as existing topic classification datasets.

This notebook aims to explore the paper and its methods in more detail, and the headings follow the paper section by section.
We go much deeper into the data than the paper, and reproduce their methods, and get similar (but slightly better) results.
A good future work would be to look into applying more modern methods on this dataset.

# Related work

A lot of the papers cited in Related Work are hard to access today, or contain almost no detail.
There are three exceptions:

* Vasileios Hatzivassiloglou and Kathleen R. McKeown. 1997. [Predicting the Semantic Orientation of Adjectives](https://aclanthology.org/P97-1023/) mines positive and negative semantic lexicons for adjectives by looking at nearby conjunctions (e.g. "fair and legitimate")
* Turney and Littman, 2002. [Unsupervised Learning of Semantic Orientation from a Hundred-Billion-Word Corpus](https://arxiv.org/abs/cs/0212012) determines whether a word is positive or negative based on whether it occurs near a positive word (like good or nice) or a negative word (like bad or poor) on the web. In particular it leveraged AltaVista's "near" search and calculates the Pointwise Mutual Information for each sentiment pair.
* Peter Turney. 2002. [Thumbs Up or Thumbs Down? Semantic Orientation Applied to Unsupervised Classification of Reviews](https://aclanthology.org/P02-1053/) was concurrent with this work and uses the approach of the previous paper to classify the sentiment of reviews from epinions.com. In particular it extracts certain POS tags, works out their semantic orientation (using "good" as the positive term and "poor" as the negative), and adds the log scores for all the words in each review.

This paper differs in that it uses more classical machine learning approaches with 1-hot encoded word features.

# The Movie Review Domain

They took reviews from the Internet Movie Database (IMDb) archive of the `rec.arts.movies.reviews`, took the reviews with a numerical or star rating and labelled the highest scored ones positive, the lowest negative, and removed the rest.

The IMDb archive no longer exists, but there are current archives of this newsgroup in [Google Groups](https://groups.google.com/g/rec.arts.movies.reviews) and the [Usenet Archives](https://www.usenetarchives.com/threads.php?id=rec.arts.movies.reviews&y=0&r=0&p=0).
Thankfully the authors released their [original data](https://www.cs.cornell.edu/people/pabo/movie-review-data/) both the raw HTML they extracted and the extracted text they used for classification.

Let's take a look at the HTML to see what they worked with

In [1]:
from urllib.request import urlretrieve
from pathlib import Path
from zipfile import ZipFile
import tarfile
import re

data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

source_html_url = 'http://www.cs.cornell.edu/people/pabo/movie-review-data/polarity_html.zip'

raw_html_path = data_dir / 'polarity_html.zip'
if not raw_html_path.exists():
    urlretrieve(source_html_url, raw_html_path)
    
raw_html_zip = ZipFile(raw_html_path)

The zipfile contains a single directory `movie` containing  around 27k review files

In [2]:
len(raw_html_zip.infolist())

27887

In [3]:
raw_html_zip.infolist()[:5]

[<ZipInfo filename='movie/0002.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=4415 compress_size=2170>,
 <ZipInfo filename='movie/0003.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=2702 compress_size=1398>,
 <ZipInfo filename='movie/0004.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=6165 compress_size=3059>,
 <ZipInfo filename='movie/0005.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=4427 compress_size=2103>,
 <ZipInfo filename='movie/0006.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=6423 compress_size=3225>]

In [4]:
raw_html_zip.infolist()[-5:]

[<ZipInfo filename='movie/9995.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=5232 compress_size=2643>,
 <ZipInfo filename='movie/9997.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=10113 compress_size=4812>,
 <ZipInfo filename='movie/9998.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=3868 compress_size=1935>,
 <ZipInfo filename='movie/9999.html' compress_type=deflate filemode='-rw-rw-rw-' file_size=3081 compress_size=1605>,
 <ZipInfo filename='movie/' filemode='drwxrwxrwx' external_attr=0x10>]

Let's have a look at one of them (that's not too long, and recent enough to be in other archives); you could also see it on the [Usenet Archives](https://www.usenetarchives.com/view.php?id=rec.arts.movies.reviews&mid=PDE5OTFGZWIxLjE1MjA0My4yNDE2MUBjYm5ld3NqLmF0dC5jb20%2B) or [Google Groups](https://groups.google.com/g/rec.arts.movies.reviews/c/Itr4BZd_EKk/m/QE0gZFPUU50J).

Note that the original was almost certainly a plaintext email; some the HTML markup (in particular the footer) would have been added by IMDB.
Note that the rating is stated *twice* in the review as a "low 0" on a scale from -4 to 4; this begins to indicate the difficulty of automatically extracting the ratings.

In [5]:
CODEC = 'ISO-8859-1'
movie_review_html = raw_html_zip.read('movie/0908.html').decode(CODEC)
print(movie_review_html)

<HTML><HEAD>
<TITLE>Review for Flight of the Intruder (1990)</TITLE>
<LINK REL="STYLESHEET" TYPE="text/css" HREF="/ramr.css">
</HEAD>
<BODY BGCOLOR="#FFFFFF" TEXT="#000000">
<H1 ALIGN="CENTER" CLASS="title"><A HREF="/Title?0099587">Flight of the Intruder (1990)</A></H1><H3 ALIGN=CENTER>reviewed by<BR><A HREF="/ReviewsBy?Mark+R.+Leeper">Mark R. Leeper</A></H3><HR WIDTH="40%" SIZE="4">
<PRE>                            FLIGHT OF THE INTRUDER
                       A film review by Mark R. Leeper
                        Copyright 1991 Mark R. Leeper</PRE>
<P>          Capsule review:  Pretty pictures, stupid story.  The
     air-war of a previous conflict is occasionally entertaining
     to watch but the plot is cliched as are most of the
     characters.  This film's only chance is to follow the current
     wave of interest in military equipment.  Rating: low 0.</P>
<P>     Had I not actually seen a copy of the book FLIGHT OF THE INTRUDER by
Stephen Coonts, I would have had a hard time 

For convenience let's define a function to read the HTML of a given movie

In [6]:
def get_movie_html(movieid):
    return raw_html_zip.read(f'movie/{movieid}.html').decode(CODEC)

In [7]:
print(get_movie_html('0908')[:1000])

<HTML><HEAD>
<TITLE>Review for Flight of the Intruder (1990)</TITLE>
<LINK REL="STYLESHEET" TYPE="text/css" HREF="/ramr.css">
</HEAD>
<BODY BGCOLOR="#FFFFFF" TEXT="#000000">
<H1 ALIGN="CENTER" CLASS="title"><A HREF="/Title?0099587">Flight of the Intruder (1990)</A></H1><H3 ALIGN=CENTER>reviewed by<BR><A HREF="/ReviewsBy?Mark+R.+Leeper">Mark R. Leeper</A></H3><HR WIDTH="40%" SIZE="4">
<PRE>                            FLIGHT OF THE INTRUDER
                       A film review by Mark R. Leeper
                        Copyright 1991 Mark R. Leeper</PRE>
<P>          Capsule review:  Pretty pictures, stupid story.  The
     air-war of a previous conflict is occasionally entertaining
     to watch but the plot is cliched as are most of the
     characters.  This film's only chance is to follow the current
     wave of interest in military equipment.  Rating: low 0.</P>
<P>     Had I not actually seen a copy of the book FLIGHT OF THE INTRUDER by
Stephen Coonts, I would have h


## Cleaned Text

And the cleaned and labelled text we'll get version 1.1 which [according to the README](https://www.cs.cornell.edu/people/pabo/movie-review-data/README.1.1) has some corrections over the version used in the paper.

In [8]:
sentiment_url = 'http://www.cs.cornell.edu/people/pabo/movie-review-data/mix20_rand700_tokens_0211.tar.gz'
sentiment_path = data_dir / 'sentiment.tar.gz'

if not sentiment_path.exists():
    urlretrieve(sentiment_url, sentiment_path)
    
urlretrieve(sentiment_url, sentiment_path)

sentiment_fh = tarfile.open(sentiment_path)

There's:

* a `diff.txt`that says what changed between versions
* a [`README`](https://www.cs.cornell.edu/people/pabo/movie-review-data/README.1.1) describing the dataset
* subfolders `neg/` and `pos/` containing negative and positive reviews

In [9]:
sentiment_fh.getnames()[:10]

['diff.txt',
 'README',
 'tokens',
 'tokens/neg',
 'tokens/neg/cv303_tok-11557.txt',
 'tokens/neg/cv000_tok-9611.txt',
 'tokens/neg/cv001_tok-19324.txt',
 'tokens/neg/cv002_tok-3321.txt',
 'tokens/neg/cv003_tok-13044.txt',
 'tokens/neg/cv004_tok-25944.txt']

In [10]:
sentiment_fh.getnames()[-10:]

['tokens/pos/cv690_tok-23617.txt',
 'tokens/pos/cv691_tok-11491.txt',
 'tokens/pos/cv692_tok-24295.txt',
 'tokens/pos/cv693_tok-16307.txt',
 'tokens/pos/cv694_tok-18628.txt',
 'tokens/pos/cv695_tok-12873.txt',
 'tokens/pos/cv696_tok-10835.txt',
 'tokens/pos/cv697_tok-29325.txt',
 'tokens/pos/cv698_tok-27735.txt',
 'tokens/pos/cv699_tok-10425.txt']

We can extract the label (pos or neg) the cross-validation id (`cvid`), and the movie id from the filename with a regular expression

In [11]:
pattern = re.compile(r'^tokens/(?P<label>[^/]+)/cv(?P<cvid>[0-9]+)_tok-(?P<movieid>[0-9]+).txt$')

pattern.match('tokens/neg/cv303_tok-11557.txt').groupdict()

{'label': 'neg', 'cvid': '303', 'movieid': '11557'}

Let's extract all the data into aligned lists

In [12]:
data = {
    'label': [],
    'cvid': [],
    'movieid': [],
    'text': []
}
    

for member in sentiment_fh:
    match = pattern.match(member.name)
    if not match:
        print('Skipping %s' % member.name)
        continue
    for k, v in match.groupdict().items():
        data[k].append(v)
    data['text'].append(sentiment_fh.extractfile(member).read().decode(CODEC).rstrip())
    
{k: len(v) for k,v in data.items()}

Skipping diff.txt
Skipping README
Skipping tokens
Skipping tokens/neg
Skipping tokens/pos


{'label': 1386, 'cvid': 1386, 'movieid': 1386, 'text': 1386}

> To avoid domination of the corpus
by a small number of prolific reviewers, we imposed
a limit of fewer than 20 reviews per author per sentiment category, yielding a corpus of 752 negative
and 1301 positive reviews, with a total of 144 reviewers represented. 
>
> ...
>
> we randomly selected 700 positive-sentiment and 700
negative-sentiment documents

We get slightly fewer likely due to the updates since it was first released

In [13]:
from collections import Counter

Counter(data['label'])

Counter({'neg': 692, 'pos': 694})

## How the text was constructed

There's no explicit mention in the paper about how the text is constructed, but we can mostly reverse engineer it.

Let's take an example and look at the HTML.

In [14]:
from IPython.display import HTML, display

In [15]:
idx = 3

In [16]:
movie_html = get_movie_html(data['movieid'][idx])

HTML(movie_html)

It seems like it only keeps the paragraph text, lowercases and then pads punctuation with spaces.

In [17]:
movie_text = data['text'][idx]

print(movie_text)

the brady bunch movie is less a motion picture than a minor pop event . forget your reruns and forget your nick at nite ; this dead-on recreation of the fabled seventies sitcom is both so exhilarating and * so disturbing that you half-expect to see the ghost of rod serling appear in the narrative . did we really act this way twenty years ago ? of course not . the genius of the brady bunch movie is how it heightens the surreality of the family unit from hell by fast-forwarding them into the future . it's 1995 and greg still calls the chicks " groovy , " and marsha still considers hair-brushing the high point of her day . bell bottoms are in , and alice has yet to move out . nothing has changed within the garish walls of brady manor and , so , the plot has the family stepping out into the real world , where they are confronted by everything from carjackers to lesbians . granted , nothing * too * serious happens here . with the exception of some heavy sexual innuendo , the brady bunch mov

Let's look at this more visually marking up the differences (we lower-case the HTML at the start to make the diff better)

In [18]:
import difflib
import html

def mark_span(text, color):
        return f'<span style="background: {color};">{html.escape(text)}</span>'

def show_diff(a, b):
    seqmatcher = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=False)
    delcolor='#f4bcbb'
    inscolor='#b0d6d6'
    
    output = []


    for tag, a0, a1, b0, b1 in seqmatcher.get_opcodes():
        if tag == 'delete':
            output.append(mark_span(a[a0:a1], color=delcolor))
        elif tag == 'insert':
            output.append(mark_span(b[b0:b1], color=inscolor))
        elif tag == 'replace':
            output.append(mark_span(a[a0:a1], color=inscolor))
            output.append(mark_span(b[b0:b1], color=delcolor))
        elif tag == 'equal':
            output.append(html.escape(a[a0:a1]))
        else:
            raise ValueError

    display(HTML(''.join(output)))
    
show_diff(movie_html.lower(), movie_text)

The first obvious thing is it extracts the main paragraph text.

One way to do this is to get all paragraphs (this isn't the only way to do it; we'll have to test whether it's correct).

In [19]:
#paragraph_text = re.match('.*?((?:<P>[^<]*</P>\s*)+)', movie_html, flags=re.DOTALL | re.IGNORECASE).group(1)
paragraph_text = ' '.join(re.findall('<P>[^<]*</P>', movie_html, flags=re.DOTALL | re.IGNORECASE))
print(paragraph_text)

<P>     THE BRADY BUNCH MOVIE is less a motion picture than a minor pop
event.  Forget your reruns and forget your Nick at Nite; this dead-on
recreation of the fabled seventies sitcom is both so exhilarating *and*
so disturbing that you half-expect to see the ghost of Rod Serling
appear in the narrative.</P> <P>     Did we really act this way twenty years ago?  Of course not.  The
genius of THE BRADY BUNCH MOVIE is how it heightens the surreality of
the Family Unit From Hell by fast-forwarding them into the future.
It's 1995 and Greg still calls the chicks "groovy," and Marsha still
considers hair-brushing the high point of her day.  Bell bottoms are
in, and Alice has yet to move out.</P> <P>     Nothing has changed within the garish walls of Brady Manor and,
so, the plot has the family stepping out into the real world, where
they are confronted by everything from carjackers to lesbians.
Granted, nothing *too* serious happens here.  With the exception of
some heavy sexual innuendo, THE

We have to remove the other tags and put spaces around punctuation.

In [20]:
show_diff(paragraph_text.lower(), movie_text)

In [21]:
# Remove HTML Tags
html_removed = re.sub('<.*?>', ' ', paragraph_text)
# Extra spaces around tokens
token_expanded = re.sub(r'([!?.*()";:,])', r' \1 ', html_removed)
# Compress multiple spaces
space_adjusted = re.sub('\s+', ' ', token_expanded).strip()

There's one remaining difference in the `*`, which is likely to do with the star rating removal logic.

In [22]:
show_diff(space_adjusted.lower(), movie_text)

We can wrap this all up in a function

In [23]:
def clean_html(movie_html):
    # Find from the first paragraph all subsequent paragraphs
    #paragraph_text = re.match('.*?((?:<P>[^<]*</P>\s*)+)', movie_html, flags=re.DOTALL | re.IGNORECASE).group(1)
    paragraph_text = ' '.join(re.findall('<P>[^<]*</P>', movie_html, flags=re.DOTALL | re.IGNORECASE))

    # Remove HTML Tags
    html_removed = re.sub('<.*?>', ' ', paragraph_text)
    # Extra spaces around tokens
    token_expanded = re.sub(r'([!?.*()";:,])', r' \1 ', html_removed)
    # Compress multiple spaces
    space_adjusted = re.sub('\s+', ' ', token_expanded).strip()
    
    return space_adjusted.lower()

In [24]:
show_diff(clean_html(movie_html), movie_text)

It's getting a bit hard to find the differences so we'll write it shorter.

In [25]:
def show_diff_short(a, b, window=5):
    seqmatcher = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=False)
    delcolor='#f4bcbb'
    inscolor='#b0d6d6'
    
    output = []


    for tag, a0, a1, b0, b1 in seqmatcher.get_opcodes():
        if tag == 'delete':
            output.append(html.unescape(a[a0-window:a0]) + mark_span(a[a0:a1], color=delcolor) + html.unescape(a[a1:a1+window]))
        elif tag == 'insert':
            output.append(html.unescape(b[b0-window:b0]) + mark_span(b[b0:b1], color=inscolor) + html.unescape(b[b1:b1+window]))
        elif tag == 'replace':
            output.append(html.unescape(a[a0-window:a0]) + mark_span(a[a0:a1], color=delcolor) + html.unescape(a[a1:a1+window]))
            output.append(html.unescape(b[b0-window:b0]) + mark_span(b[b0:b1], color=inscolor) + html.unescape(b[b1:b1+window]))
        elif tag == 'equal':
            pass
        else:
            raise ValueError

    display(HTML('<p>'.join(output)))
    
show_diff_short(clean_html(movie_html), movie_text)

### Checking examples

Let's look through some examples to see what's different.

It looks like we're missing the author's details (which may be a good thing)

In [26]:
idx = 1
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

We missed square bracket punctuation and another author name

In [27]:
idx = 2
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [28]:
idx = 3
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

Again missing author details

In [29]:
idx = 4
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

Missing author details, and also the rating scale (which again seem irrelevant).

In [30]:
idx = 5
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

### Making some corrections

It looks like we're missing URLs because we're not capturing the `<A>` tags, so we'll allow those too, as well as Horizontal Rules

In [31]:
def clean_html(movie_html):
    # Find from the first paragraph all subsequent paragraphs
    #paragraph_text = re.match('.*?((?:<P>[^<]*</P>\s*)+)', movie_html, flags=re.DOTALL | re.IGNORECASE).group(1)
    paragraph_text = ' '.join(re.findall('<P>(?:[^<]|</?A|<HR)*</P>', movie_html, flags=re.DOTALL | re.IGNORECASE))

    # Remove HTML Tags
    html_removed = re.sub('<.*?>', ' ', paragraph_text)
    # Extra spaces around tokens
    token_expanded = re.sub(r'([+!?.*()";:,])', r' \1 ', html_removed)
    # Compress multiple spaces
    space_adjusted = re.sub('\s+', ' ', token_expanded).strip()
    
    return space_adjusted.lower()

In [32]:
idx = 1
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [33]:
idx = 2
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [34]:
idx = 3
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [35]:
idx = 4
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [36]:
idx = 5
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [37]:
idx = 6
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

There are cases here we are getting cast names and links that were not in the processed dataset.

In [38]:
idx = 7
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

In [39]:
idx = 8
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

Here's a really interesting case where their extracted text loses a lot of the review.

In [40]:
idx = 9
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

The review stops part way through the list

In [41]:
print(data['text'][idx])

starring : tommy lee jones ( sam gerard ) , wesley snipes ( various character names ) , joe pantoliano ( cosmo renfro ) , robert downey jr . ( john royce ) , irène jacob ( marie ) , tom wood ( noah newman ) , daniel roebuck ( biggs ) , kate nelligan ( walsh ) directed by : stuart baird , written by : john poguerated pg-13 by the mpaa for violence , gore , and strong languagei'll lay out the basics first . sam gerard , the chief deputy u . s . marshal from the fugitive ( 1993 ) , and his team of crack deputies , are back . another man has fled from the clutches of the state , and gerard must chase this new fugitive down . but what's the story this time ? why is he running ? did he do what he was accused ? if not , who did ? who's doing the cover-up ? why ? and so on . all the stuff you love from your traditional action/suspense/mystery flicks , but with better characters ( especially jones' oscar-winning gerard role ) and better action set-pieces , pushing it into the above-average fun-

The line after "copy of THE FUGITIVE." is marked as a `<PRE>`, which seems to be some issue with how IMDb extracted it (there's no indication this should be the case in the [Google Groups Version](https://groups.google.com/g/rec.arts.movies.reviews/c/cKNMJP1GVVo/m/IesbVByox4MJ))

It looks like this is somehow causing the text parsing to stop, so there are likely some heuristics around `<PRE>`.

In [42]:
HTML(get_movie_html(data['movieid'][idx]))

They have also removed the review rating we've captured

In [43]:
idx = 10
show_diff_short(clean_html(get_movie_html(data['movieid'][idx])),
          data['text'][idx])

Although they're not perfect at this and sometimes a star rating sneaks through

In [44]:
print(data['text'][1088][-50:])

 how good it was . the wood earns my * * * stars .


### Upshot

We haven't quite reconstructed their version, but there are some things we can note:

* They often include authorship information which probably should be masked or excluded
* They sometimes miss large parts of the review
* They do try to remove the star ratings data

## How the labels were constructed

As well as extracting the clean text they also needed to extract the star rating which is often in an inconsistent format.
Here are some real examples

In [45]:
star_examples = {
    'RATING: *1/2 OUT OF ****': (1.5, 4),
    '*** (out of ****)': (3, 4),
    '1 star (out of five stars)': (1,5),
    '* (out of ****)': (1,4),
    '* (out of 4) - a poor movie': (1,4),
    '*** 1/2': (3.5, None),
    'My Rating: ** out of *****': (2,5),
    '* (out of * * * * )': (1,4),
    'Psychosis Rating:  4/10': (4,10),
    'RATING=**** OUT OF *****': (4,5),
    '* * *': (3, None),
    'RATINGS:  7 / 10 --> Good movie': (7,10),
    '*** (out of 4)': (3,4),
    'RATING:  ***1/2': (3.5, None),
    'Rating: 1.0 stars out of 5.0 stars': (1,5),
    '***1/2 (of 4)': (3.5, 4),
    'Two out of four stars': (2, 4),
    '1 1/2* out of * * * *': (1.5, 4)
    }


One way to do this is to systematically rewrite the string into something easy to parse.

For numbers expressed as words we can substitue them with the corresponding numbers.

In [46]:
number_words = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']

def replace_number_words(s):
    for i, word in enumerate(number_words):
        s = re.sub(fr'\b{word}\b', str(i), s, flags=re.IGNORECASE)
    return s

In [47]:
replace_number_words('Two out of four stars')

'2 out of 4 stars'

For stars we can count them; first we find them (with maybe a fraction)

In [48]:
match = re.match(r'((?:\*\s*)+)(\d+\s*/\d+\*?)?', '* * * 1/2')
match.groups()

('* * * ', '1/2')

Then we can extract both the stars and the fraction

In [49]:
stars, frac = match.groups()

Count the stars

In [50]:
len(stars.replace(' ', ''))

3

In [51]:
numerator, denominator = re.match('(\d+)\s*/(\d+)', frac).groups()

In [52]:
float(numerator) / float(denominator)

0.5

We can wrap this in a function and return a string to substitute in the string

In [53]:
def stars_to_number_str(match):
    stars, frac = match.groups()
    star_count = len(re.sub('\s+', '', stars))
    if frac:
        numerator, denominator = re.match('(\d+)\s*/(\d+)', frac).groups()
        fraction = float(numerator) / float(denominator)
        if fraction >= 1:
            raise ValueError("Unexpected fraction %s in %s" % (frac, match))
        star_count += fraction
    return str(star_count) + ' '

In [54]:
stars_to_number_str(match)

'3.5 '

In [55]:
def replace_star_counts(s):
    return re.sub(r'((?:\*\s*)+)(\d+\s*/\d+)?', stars_to_number_str, s)

In [56]:
replace_star_counts('*1/2 (out of ****)')

'1.5  (out of 4 )'

And finally let's normalise the totals to use the fraction notation

In [57]:
def replace_out_of(s):
    return re.sub(r'\(?(\s*stars?)?\s*\(?(\s*out)?\s*of', '/', s, flags=re.IGNORECASE)

In [58]:
re.match(         r'\(?(\s*stars?)?\s*\(?(\s*out)?\s*of', 'star (out of')

<re.Match object; span=(0, 12), match='star (out of'>

In [59]:
replace_out_of('star (out of)')

'/)'

In [60]:
r'\(?(\s*stars?)?\s*\(?(\s*out)?\s*of' == r'\(?(\s*stars?)?\s*\(?(\s*out)?\s*of'

True

In [61]:
replace_out_of('1.5 (out of 4)')

'1.5/ 4)'

Let's compose these all together

In [62]:
def normalise_ratings(s):
    s = replace_number_words(s)
    s = replace_star_counts(s)
    s = replace_out_of(s)
    return s

This results in ratings having a simple numeric form we can parse

In [63]:
for text in star_examples:
    print(text, ';', normalise_ratings(text))

RATING: *1/2 OUT OF **** ; RATING: 1.5/ 4 
*** (out of ****) ; 3/ 4 )
1 star (out of five stars) ; 1/ 5 stars)
* (out of ****) ; 1/ 4 )
* (out of 4) - a poor movie ; 1/ 4) - a poor movie
*** 1/2 ; 3.5 
My Rating: ** out of ***** ; My Rating: 2/ 5 
* (out of * * * * ) ; 1/ 4 )
Psychosis Rating:  4/10 ; Psychosis Rating:  4/10
RATING=**** OUT OF ***** ; RATING=4/ 5 
* * * ; 3 
RATINGS:  7 / 10 --> Good movie ; RATINGS:  7 / 10 --> Good movie
*** (out of 4) ; 3/ 4)
RATING:  ***1/2 ; RATING:  3.5 
Rating: 1.0 stars out of 5.0 stars ; Rating: 1.0/ 5.0 stars
***1/2 (of 4) ; 3.5/ 4)
Two out of four stars ; 2/ 4 stars
1 1/2* out of * * * * ; 1 1/21/ 4 


We can then parse these quite simply; taking the leftmost match

In [64]:
def parse_normalised_ratings(s):
    matches = re.findall(r'\b((?:\d|10)(?:\.\d*)?)(?:\s*/\s*((?:\d|10)(?:\.\d*)?))?\b', s)
    if not matches:
        return None
    score, total = matches[0]
    
    return (float(score), float(total) if total else None)

In [65]:
errors = 0
total = len(star_examples)
for text, expected in star_examples.items():
    result = parse_normalised_ratings(normalise_ratings(text))
    if result != expected:
        errors += 1
        print(f'In {text} got {result} but expected {expected}')

print(f'{total-errors} / {total} correct')

In 1 1/2* out of * * * * got (1.0, None) but expected (1.5, 4)
17 / 18 correct


Now do these work on actual reviews which may contain a lot of other text?

In [66]:
movie_html = get_movie_html(data['movieid'][0])

Consider this example:

In [67]:
parse_normalised_ratings(normalise_ratings(movie_html))

(4.0, None)

The 4 comes from a size attribute in a HTML tag on the end of line 5; we should also consider HTML tags.

We actually want line 8 (1.5,  4)

In [68]:
for i, line in enumerate(movie_html[:1000].split('\n')):
    print(i, line)

0 <HTML><HEAD>
1 <TITLE>Review for Wild Things (1998)</TITLE>
2 <LINK REL="STYLESHEET" TYPE="text/css" HREF="/ramr.css">
3 </HEAD>
4 <BODY BGCOLOR="#FFFFFF" TEXT="#000000">
5 <H1 ALIGN="CENTER" CLASS="title"><A HREF="/Title?0120890">Wild Things (1998)</A></H1><H3 ALIGN=CENTER>reviewed by<BR><A HREF="/ReviewsBy?James+Berardinelli">James Berardinelli</A></H3><HR WIDTH="40%" SIZE="4">
6 <PRE>WILD THINGS</PRE>
7 <PRE>A Film Review by James Berardinelli</PRE>
8 <PRE>RATING: *1/2 OUT OF ****</PRE>
9 <P>United States, 1998
10 U.S. Release Date: 3/20/98 (wide)
11 Running Length: 1:48
12 MPAA Classification: R (Male full-frontal nudity, female topless nudity, 
13       graphic sexual activity, frequent profanity, violence)
14 Theatrical Aspect Ratio: 2.35:1</P>
15 <P>Cast: Kevin Bacon, Matt Dillon, Neve Campbell, Theresa Russell, 
16       Denise Richards, Daphne Rubin-Vega, Carrie Snodgress, Jeff Perry, 
17       Robert Wagner, Bill Murray
18 Director: John McNaughton
19 Producers: Steven A. J

In [69]:
def remove_tags(s):
    return re.sub('<[^>]+>', ' ', s)

def get_rating(s):
    return parse_normalised_ratings(normalise_ratings(remove_tags(s)))

Then we get the correct result

In [70]:
get_rating(movie_html)

(1.5, 4.0)

There are still some things wrong; this movie apparently has a rating of 7 which seems suspect

In [71]:
movie_html = get_movie_html(data['movieid'][7])
get_rating(movie_html)

(7.0, None)

But reading into the text it's actually from the string "Lucky 7"; the rating is right down the bottom

In [72]:
HTML(movie_html)

We could do better with some more heuristics; most of the time the label is in a `<PRE>` tag, it's often in a sentence with the word RATING etc.

In effect extracting the rating is itself another NLP task; you can build a rule system for it but it quickly gets tricky to cover all the cases.
It could be interesting to build a system to find the rating string and convert it into a number.

### Testing sentiment extration logic

Let's take this further; from the README

> - For ratings specified in stars, we assume a maximum of four stars unless the author specified otherwise (e.g. "*** out of *****");
> - For ratings specified in numbers, the maximum rating must be specified explicitly.
> - With a five-star system (or compatible number systems):
    four stars and up are considered positive, 
	two stars and below are considered negative; 
> - With a four-star system (or compatible number system):
	three stars and up are considered positive, 
	one star and below are considered negative. 
>
> We attempted to recognize half stars, but they are specified in an
especially free way, which makes them difficult to recognize.  Hence,
we may lose a half star occasionally; but this only results in 2.5
stars in five star system (1.5 stars in four star system) being
categorized as negative, which is still reasonable.

In [73]:
def get_binary_classification(movie_html, count_halves=True):
    rating, max_rating = get_rating(movie_html)
    
    # we assume a maximum of four stars unless the author specified otherwise
    if max_rating is None or max_rating == 0:
        max_rating = 4
        
    # We attempted to recognize half stars
    if not count_halves:
        rating = int(rating)
    
    # For e.g. /10 we will divide to make it out of 5
    if max_rating % 5 == 0:
        divisor = max_rating // 5   
        rating /= divisor
        max_rating /= divisor
        
    if max_rating == 5:
        if rating >= 4:
            return 'pos'
        elif rating <= 2:
            return 'neg'
        else:
            return 'neutral'
        
    if max_rating == 4:
        if rating >= 3:
            return 'pos'
        elif rating <= 1:
            return 'neg'
        else:
            return 'neutral'
        
    return 'error'

There are a lot of neutral predictions

In [74]:
extracted_labels = [get_binary_classification(get_movie_html(movie_id)) for movie_id in data['movieid']]
Counter(zip(extracted_labels, data['label'])).most_common()

[(('pos', 'pos'), 459),
 (('neg', 'neg'), 346),
 (('neutral', 'neg'), 268),
 (('neg', 'pos'), 129),
 (('neutral', 'pos'), 106),
 (('pos', 'neg'), 67),
 (('error', 'neg'), 11)]

In [75]:
mismatches = [(movie_id, extracted, label) for movie_id, extracted, label in zip(data['movieid'], extracted_labels, data['label']) if extracted != label]
f'Error rate: {len(mismatches) / len(extracted_labels):0.0%}'

'Error rate: 42%'

And they are often coming from half stars

In [76]:
get_rating(get_movie_html(mismatches[0][0]))

(1.5, 4.0)

We get a more consistent result when we drop the half stars

In [77]:
extracted_labels = [get_binary_classification(get_movie_html(movie_id), count_halves=False) for movie_id in data['movieid']]
Counter(zip(extracted_labels, data['label'])).most_common()

[(('neg', 'neg'), 556),
 (('pos', 'pos'), 459),
 (('neg', 'pos'), 129),
 (('neutral', 'pos'), 106),
 (('pos', 'neg'), 67),
 (('neutral', 'neg'), 58),
 (('error', 'neg'), 11)]

In [78]:
mismatches = [(movie_id, extracted, label) for movie_id, extracted, label in zip(data['movieid'], extracted_labels, data['label']) if extracted != label]
f'Error rate: {len(mismatches) / len(extracted_labels):0.0%}'

'Error rate: 27%'

At a glance it seems it's often because we're picking up other things as text.
We'd have to work a bit harder to get the correct result.

In [79]:
get_rating(get_movie_html(mismatches[0][0]))

(7.0, None)

## Data Lengths

These reviews can be quite long, and the tokenization of punctuation is quite aggressive; how long are the actual tokens?

In [80]:
lengths = [len(text.split()) for text in data['text']]

In [81]:
def median(l):
    return sorted(l)[len(l)//2]

def mean(l):
    return sum(l)/len(l)

The median length of reviews is around 700

In [82]:
median(lengths), mean(lengths)

(700, 756.8600288600288)

Note that negative reviews tend to be a little shorter than positive reviews.

In [83]:
neg_lengths = [x for x, l in zip(lengths, data['label']) if l == 'neg']
median(neg_lengths), mean(neg_lengths)

(681, 715.5664739884393)

In [84]:
pos_lengths = [x for x, l in zip(lengths, data['label']) if l == 'pos']
median(pos_lengths), mean(pos_lengths)

(735, 798.0345821325649)

We can use this to get a few percentage points without even looking at the data

In [85]:
preds = ['pos' if l > 730 else 'neg' for l in lengths]

In [86]:
def accuracy(preds, actuals):
    if len(preds) != len(actuals):
        raise ValueError('Expected same length input')
    return mean([p==a for p,a in zip(preds, actuals)])

accuracy(preds, data['label'])

0.5440115440115441

# A Closer Look at the Problem

From the paper they set a benchmark using manual lists of positive and negative words:

> One might also suspect that there are certain words
people tend to use to express strong sentiments, so
that it might suffice to simply produce a list of such
words by introspection and rely on them alone to
classify the texts.
>
>To test this latter hypothesis, we asked two graduate students in computer science to (independently)
choose good indicator words for positive and negative sentiments in movie reviews.

Extracting these from Figure 1

In [87]:
positive1 = 'dazzling, brilliant, phenomenal, excellent, fantastic'.split(', ')
positive1

['dazzling', 'brilliant', 'phenomenal', 'excellent', 'fantastic']

In [88]:
negative1 = 'suck, terrible, awful, unwatchable, hideous'.split(', ')
negative1

['suck', 'terrible', 'awful', 'unwatchable', 'hideous']

In [89]:
positive2 = 'gripping, mesmerizing, riveting, spectacular, cool, ' \
            'awesome, thrilling, badass, excellent, moving, exciting'.split(', ')
positive2

['gripping',
 'mesmerizing',
 'riveting',
 'spectacular',
 'cool',
 'awesome',
 'thrilling',
 'badass',
 'excellent',
 'moving',
 'exciting']

In [90]:
negative2 = 'bad, cliched, sucks, boring, stupid, slow'.split(', ')
negative2

['bad', 'cliched', 'sucks', 'boring', 'stupid', 'slow']

> We then converted their responses into simple decision
procedures that essentially count the number of the
proposed positive and negative words in a given document. 

We can build a small classifier to do this.
When there's a tie we need to decide how to break it with a "default".

From the paper

> Note that the tie rates — percentage of documents where the two sentiments
were rated equally likely — are quite high (we chose
a tie breaking policy that maximized the accuracy of
the baselines)

We need to look at the data for this.

In [91]:
idx2cat = ['neg', 'pos']
cat2idx = {'neg': 0, 'pos': 1}

In [92]:
class NotFittedException(Exception):
    pass


class MatchCountClassifier:
    def __init__(self, positive, negative):
        self.positive = positive
        self.negative = negative
        self.default = None
        self.ties = None
        
    def _score(self, tokens):
        """Return number of positive words - number of negative words in token"""
        pos_count = len([t for t in tokens if t in self.positive])
        neg_count = len([t for t in tokens if t in self.negative])
        return pos_count - neg_count
        
    def fit(self, X, y):
        """Find default that maximises """
        scores = [self._score(tokens) for tokens in X]
        self.ties = len([x for x in scores if x==0]) / len(scores)
        
        pred_pos_default = [1 if x >= 0 else 1 for x in scores]
        pred_neg_default = [0 if x <= 0 else 0 for x in scores]
        
        if accuracy(pred_pos_default, y) >= accuracy(pred_neg_default, y):
            self.default = 1
        else:
            self.default = 0
            
        return self
        
    def predict(self, X):
        if self.default is None:
            raise NotFittedException()
        scores = [self._score(tokens) for tokens in X]
        
        return [1 if score > 0 else 0 if score < 0 else self.default for score in scores]

Let's test our class

In [93]:
mcc_test = MatchCountClassifier(['happy'], ['sad'])

In [94]:
X_test = [['happy'], ['sad'], ['happy', 'sad']]
y_test_1 = [1, 0, 1]
y_test_2 = [1, 0, 0]

In [95]:
mcc_test.fit(X_test, y_test_1)
assert mcc_test.default == 1

In [96]:
assert mcc_test.predict([['sad'], []]) == [0, 1]

In [97]:
assert mcc_test.ties == 1/3

In [98]:
mcc_test.fit(X_test, y_test_2)
assert mcc_test.default == 0

In [99]:
assert mcc_test.predict([['sad'], []]) == [0, 0]

## Human Baselines

Let's get the tokens and labels

In [100]:
X = [text.split() for text in data['text']]
y = [cat2idx[l] for l in data['label']]

The ties and accuracy matches table 1 for Human 1

In [101]:
mcc1 = MatchCountClassifier(positive1, negative1)
mcc1.fit(X, y)

print(f'''Human 1
Ties: {mcc1.ties:0.0%}
Accuracy: {accuracy(mcc1.predict(X), y):0.0%}''')

Human 1
Ties: 75%
Accuracy: 56%


For Human 2 we get accuracy 1 percentage point higher than the paper; likely due to corrections to the dataset.

In [102]:
mcc2 = MatchCountClassifier(positive2, negative2)
mcc2.fit(X, y)

print(f'''Human 2
Ties: {mcc2.ties:0.0%}
Accuracy: {accuracy(mcc2.predict(X), y):0.0%}''')

Human 2
Ties: 39%
Accuracy: 65%


They also provide a third baseline in table 2 using statistics from the dataset

> Based on a very
preliminary examination of frequency counts in the
entire corpus (including test data) plus introspection,
we created a list of seven positive and seven negative
words (including punctuation), shown in Figure 2.

In [103]:
positive3 = 'love, wonderful, best, great, superb, still, beautiful'.split(', ')
positive3

['love', 'wonderful', 'best', 'great', 'superb', 'still', 'beautiful']

In [104]:
negative3 = 'bad, worst, stupid, waste, boring, ?, !'.split(', ')
negative3

['bad', 'worst', 'stupid', 'waste', 'boring', '?', '!']

Again we get an accuracy 1% higher

In [105]:
mcc3 = MatchCountClassifier(positive3, negative3)
mcc3.fit(X, y)

print(f'''Human 3 + stats
Ties: {mcc3.ties:0.0%}
Accuracy: {accuracy(mcc3.predict(X), y):0.0%}''')

Human 3 + stats
Ties: 15%
Accuracy: 70%


## Could we do better?

An obvious strategy would be to combine the lists; they are mostly disjoint.

However that doesn't improve our accuracy at all over Human 3

In [106]:
mcc_all = MatchCountClassifier(set(positive1 + positive2 + positive3),
                               set(negative1 + negative2 + negative3))

mcc_all.fit(X, y)

print(f'''Combined Humans
Ties: {mcc_all.ties:0.0%}
Accuracy: {accuracy(mcc_all.predict(X), y):0.0%}''')

Combined Humans
Ties: 13%
Accuracy: 70%


Another resource that was available at the time was the [Harvard General Inquirer](https://inquirer.sites.fas.harvard.edu/) lexicon which tags words with a `positiv` or `negativ` sentiment, among many other classifications.

I can't find an official source for the lexicon, but there's a version inside the [pysentiment library](https://github.com/nickderobertis/pysentiment/) (which may be different to what was available at the time).

In [107]:
import csv

harvard_inquirer_url = 'https://raw.githubusercontent.com/nickderobertis/pysentiment/master/pysentiment2/static/HIV-4.csv'
harvard_inquirer_path = data_dir / 'HIV-4.csv'
if not harvard_inquirer_path.exists():
    urlretrieve(harvard_inquirer_url, harvard_inquirer_path)

with open(harvard_inquirer_path) as f:
    harvard_inquirer_data = list(csv.DictReader(f))
    
harvard_inquirer_data[1]

{'Entry': 'ABANDON',
 'Source': 'H4Lvd',
 'Positiv': '',
 'Negativ': 'Negativ',
 'Pstv': '',
 'Affil': '',
 'Ngtv': 'Ngtv',
 'Hostile': '',
 'Strong': '',
 'Power': '',
 'Weak': 'Weak',
 'Submit': '',
 'Active': '',
 'Passive': '',
 'Pleasur': '',
 'Pain': '',
 'Feel': '',
 'Arousal': '',
 'EMOT': '',
 'Virtue': '',
 'Vice': '',
 'Ovrst': '',
 'Undrst': '',
 'Academ': '',
 'Doctrin': '',
 'Econ@': '',
 'Exch': '',
 'ECON': '',
 'Exprsv': '',
 'Legal': '',
 'Milit': '',
 'Polit@': '',
 'POLIT': '',
 'Relig': '',
 'Role': '',
 'COLL': '',
 'Work': '',
 'Ritual': '',
 'SocRel': '',
 'Race': '',
 'Kin@': '',
 'MALE': '',
 'Female': '',
 'Nonadlt': '',
 'HU': '',
 'ANI': '',
 'PLACE': '',
 'Social': '',
 'Region': '',
 'Route': '',
 'Aquatic': '',
 'Land': '',
 'Sky': '',
 'Object': '',
 'Tool': '',
 'Food': '',
 'Vehicle': '',
 'BldgPt': '',
 'ComnObj': '',
 'NatObj': '',
 'BodyPt': '',
 'ComForm': '',
 'COM': '',
 'Say': '',
 'Need': '',
 'Goal': '',
 'Try': '',
 'Means': '',
 'Persist': 

We can extract the positive and negative entries

In [108]:
positive_hi = [i['Entry'].lower() for i in harvard_inquirer_data if i['Positiv']]
positive_hi[:5], positive_hi[-5:], len(positive_hi)

(['abide', 'ability', 'able', 'abound', 'absolve'],
 ['worth-while', 'worthiness', 'worthy', 'zenith', 'zest'],
 1915)

In [109]:
negative_hi = [i['Entry'].lower() for i in harvard_inquirer_data if i['Negativ']]
negative_hi[:5], negative_hi[-5:], len(negative_hi)

(['abandon', 'abandonment', 'abate', 'abdicate', 'abhor'],
 ['wrongful', 'wrought', 'yawn', 'yearn', 'yelp'],
 2291)

This has fewer ties but actually a lower accuracy than human 3.

(Technically we should use stemming with Harvard Inquirer but it won't improve matters here)

In [110]:
mcc_hi = MatchCountClassifier(set(positive_hi), set(negative_hi))

mcc_hi.fit(X, y)

print(f'''Harvard Inquirer
Ties: {mcc_hi.ties:0.0%}
Accuracy: {accuracy(mcc_hi.predict(X), y):0.0%}''')

Harvard Inquirer
Ties: 6%
Accuracy: 63%


## Error Analysis

In [111]:
yhat = mcc3.predict(X)
scores = [mcc3._score(row) for row in X]

In [112]:
correct = [yi==yhati for yi, yhati in zip(y, yhat)]
mean(correct)

0.6984126984126984

In [113]:
incorrect_idx = [i for i, c in enumerate(correct) if not c]
len(incorrect_idx)

418

In [114]:
for score, count in Counter(scores[i] for i in incorrect_idx).most_common():
    print(score, '\t', count, '\t', f'{count/len(incorrect_idx):0.2%}')

0 	 84 	 20.10%
-1 	 67 	 16.03%
1 	 61 	 14.59%
-2 	 51 	 12.20%
2 	 37 	 8.85%
-3 	 30 	 7.18%
-4 	 16 	 3.83%
3 	 13 	 3.11%
-5 	 11 	 2.63%
-6 	 10 	 2.39%
4 	 8 	 1.91%
5 	 7 	 1.67%
6 	 6 	 1.44%
-9 	 4 	 0.96%
-7 	 4 	 0.96%
-10 	 3 	 0.72%
-8 	 2 	 0.48%
17 	 1 	 0.24%
-15 	 1 	 0.24%
-12 	 1 	 0.24%
-11 	 1 	 0.24%


Look at most extreme cases

In [115]:
very_wrong_idx = [i for i in incorrect_idx if abs(scores[i]) >= 7]

len(very_wrong_idx)

17

In [116]:
def markup_html_words(text, words, color):
    word_pattern = '|'.join([r'\b' + re.escape(word) + r'\b' if len(word) > 1 else re.escape(word) for word in words])
    return re.sub(fr"({word_pattern})(?![^<]*>)", lambda match: mark_span(match.group(1), color), text, flags=re.IGNORECASE)
    
def markup_sentiment(text, positive=positive3, negative=negative3):
    text = markup_html_words(text, positive, "lightgreen")
    text = markup_html_words(text, negative, "orange")
    return text

In [117]:
def show_index(idx):
    movieid = data['movieid'][idx]
    print(f'Movie: {movieid}, Label: {data["label"][idx]}, Score: {scores[idx]}')
    display(HTML(markup_sentiment(get_movie_html(movieid))))

This movie is labelled as negative, despite being 3 out of 4 stars.

The author is a Woody Allen fan and it wasn't his favorite Woody Allen film but it's still pretty good.

This is a mislabelling.

In [118]:
show_index(very_wrong_idx[0])

Movie: 15970, Label: neg, Score: 17


Interestingly 15970 one of the "corrections" pos->neg

In [119]:
print()
diff_txt = sentiment_fh.extractfile(sentiment_fh.getmember('diff.txt')).read().decode(CODEC).rstrip()
print(diff_txt)


== Changes made ==

mix20_rand700_tokens_cleaned.zip 
-> mix20_rand700_tokens_0211.tar.gz

Removed : (non-English/incomplete reviews)

pos/cv037_tok-11720.txt
pos/cv206_tok-12590.txt
pos/cv263_tok-10033.txt
pos/cv365_tok-21785.txt
pos/cv400_tok-11748.txt
pos/cv528_tok-12960.txt
pos/cv627_tok-14423.txt

neg/cv059_tok-8583.txt
neg/cv111_tok-11625.txt
neg/cv193_tok-28093.txt
neg/cv216_tok-27832.txt
neg/cv219_tok-11130.txt
neg/cv423_tok-10742.txt
neg/cv592_tok-10894.txt


Moved: (based on Nathan's judgement when he read the review,
sometimes different from the original author's own rating,
as listed below)

neg -> pos:
cv279_tok-23947.txt	*1/2, but reads positive
cv346_tok-24609.txt 	misclassification
cv375_tok-0514.txt  	misclassification
cv389_tok-8969.txt 	misclassification
cv425_tok-8417.txt 	several reviews together
cv518_tok-11610.txt 	misclassification

pos -> neg:
cv017_tok-29801.txt 	*** Average, hits and misses 
cv352_tok-15970.txt 	(out of 4): *** 
cv375_tok-21437.txt 	* * * - 

This is an example of overindexing on punctuation; there are a lot of exclamation marks.

In [120]:
show_index(very_wrong_idx[1])

Movie: 13401, Label: pos, Score: -15


Again overindexing on punctuation

In [121]:
show_index(very_wrong_idx[2])

Movie: 19119, Label: pos, Score: -9


Punctuation

In [122]:
show_index(very_wrong_idx[3])

Movie: 11236, Label: pos, Score: -10


Punctuation

In [123]:
show_index(very_wrong_idx[4])

Movie: 12756, Label: pos, Score: -10


Punctuation...

In [124]:
show_index(very_wrong_idx[5])

Movie: 10406, Label: pos, Score: -10


A negation (not really too bad) and a lot of question marks

In [125]:
show_index(very_wrong_idx[6])

Movie: 7574, Label: pos, Score: -9


Almost all the words picked up are not actually sentiment ("PLOT: ... tries his best", "bad guy", "love interests")

In [126]:
show_index(very_wrong_idx[7])

Movie: 28980, Label: pos, Score: -9


# Machine Learning Methods

We'll now use traditional machine learning methods.
For showing the different methods we'll use a small vocabulary from the human baseline.

To keep this section reasonable length we'll use `sklearn` implementations of the methods.

In [127]:
vocab = positive3 + negative3
vocab

['love',
 'wonderful',
 'best',
 'great',
 'superb',
 'still',
 'beautiful',
 'bad',
 'worst',
 'stupid',
 'waste',
 'boring',
 '?',
 '!']

We'll create a Feature vector from this vocabulary for each document

In [128]:
word_counts = [Counter(word for word in doc if word in vocab) for doc in X]

X_feature = [[row[word] for word in vocab] for row in word_counts]

dict(zip(vocab, X_feature[0]))

{'love': 0,
 'wonderful': 0,
 'best': 0,
 'great': 1,
 'superb': 0,
 'still': 1,
 'beautiful': 0,
 'bad': 0,
 'worst': 0,
 'stupid': 0,
 'waste': 0,
 'boring': 0,
 '?': 1,
 '!': 0}

And split it into train and test sets by the cvid

In [129]:
X_train = [row for row, cvid in zip(X_feature, data['cvid']) if int(cvid) // 233 < 2]
X_test = [row for row, cvid in zip(X_feature, data['cvid']) if int(cvid) // 233 >= 2]

y_train = [row for row, cvid in zip(y, data['cvid']) if int(cvid) // 233 < 2]
y_test = [row for row, cvid in zip(y, data['cvid']) if int(cvid) // 233 >= 2]

len(X_train), len(X_test), len(y_train), len(y_test)

(921, 465, 921, 465)

## Naive Bayes

The text states they use Naive Bayes with add-1 smoothing (so in sklearn `alpha=1.0`):

In [130]:
from sklearn.naive_bayes import MultinomialNB


nb = MultinomialNB(alpha=1.0)

nb.fit(X_train, y_train)

accuracy(nb.predict(X_test), y_test)

0.7161290322580646

There is another way to do Naive Bayes where each word is taken as an independent feature, but it tends to be worse for NLP.

In [131]:
from sklearn.naive_bayes import BernoulliNB


nb = BernoulliNB(binarize=1.0, alpha=1.0)
nb.fit(X_train, y_train)
accuracy(nb.predict(X_test), y_test)

0.6666666666666666

In [132]:
nb = MultinomialNB(alpha=1.0)

## Maximum Entropy

Maximum Entropy is an old NLP term for Logistic Regression

> We use ten iterations of the improved iterative scaling algorithm (Della Pietra et al., 1997) for parameter training (this was a sufficient number of iterations for convergence of training-data accuracy), together with a Gaussian prior to prevent overfitting (Chen and Rosenfeld, 2000).

A Gaussian Prior is equivalent to an L2 penalty, but they don't specify the size of the prior and I can't access the referenced paper.
I'll stick to the default in sklearn of `1.0` (the solver shouldn't matter much).

In [133]:
from sklearn.linear_model import LogisticRegression

me = LogisticRegression(penalty='l2', solver='liblinear', C=1.0)

me.fit(X_train, y_train)
accuracy(me.predict(X_test), y_test)

0.6946236559139785

Note that the amount of regularization can actually matter.
Ideally we'd keep a small holdout set for hyperparameter tuning, but I'll stick to the methods in the original.

In [134]:
me = LogisticRegression(penalty='l2', solver='liblinear', C=0.5)

me.fit(X_train, y_train)
accuracy(me.predict(X_test), y_test)

0.7075268817204301

In [135]:
me = LogisticRegression(penalty='l2', solver='liblinear', C=1.0)

## Support Vector Machines

>  We used Joachim’s (1999) SVM light package for training and testing, with all parameters set to their default values, after first length-normalizing the document vectors, as is standard (neglecting to normalize generally hurt performance slightly).

There's little detail as to the hyperparameters again, so I'll use the default.

In [136]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline

svm = Pipeline([('norm', Normalizer(norm='l2')),
                ('svc', LinearSVC(penalty='l2', loss='squared_hinge', C=1.0))])

svm.fit(X_train, y_train)
accuracy(svm.predict(X_test), y_test)

0.7290322580645161

I'm not clear on what length-normalizing is, but L2 normalizing looks like it works better than L1

In [137]:
svm = Pipeline([('norm', Normalizer(norm='l1')),
                ('svc', LinearSVC(penalty='l2', loss='squared_hinge', C=1.0))])

svm.fit(X_train, y_train)
accuracy(svm.predict(X_test), y_test)

0.7118279569892473

And this works better than not normalizing it at all

In [138]:
svm = LinearSVC(penalty='l2', loss='squared_hinge', C=1.0)

svm.fit(X_train, y_train)
accuracy(svm.predict(X_test), y_test)

/home/eross/mambaforge/envs/pang_lee_2003/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.6967741935483871

As with Maximum Entropy it's sensitive to the amount of regularizaiton.

In [139]:
svm = Pipeline([('norm', Normalizer(norm='l2')),
                ('svc', LinearSVC(penalty='l2', loss='squared_hinge', C=2.0))])

svm.fit(X_train, y_train)
accuracy(svm.predict(X_test), y_test)

0.7311827956989247

We'll reset everything to the defaults

In [140]:
svm = Pipeline([('norm', Normalizer(norm='l2')),
                ('svc', LinearSVC(penalty='l2', loss='squared_hinge', C=1.0))])

# Evaluation

## Experimental Set-up


In [141]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

> we randomly selected 700 positive-sentiment and 700 negative-sentiment documents

In [142]:
Counter(y)

Counter({0: 692, 1: 694})

> We then divided this data into three equal-sized folds, maintaining balanced class distributions in each fold. 

In [143]:
folds = [[idx for idx, cvid in enumerate(data['cvid']) if int(cvid) // 233 == i] for i in range(3)]

[len(f) for f in folds]

[459, 462, 463]

In [144]:
cv = [(folds[0] + folds[1], folds[2]),
      (folds[0] + folds[2], folds[1]),
      (folds[1] + folds[2], folds[0]),
     ]

> One unconventional step we took was to attempt
to model the potentially important contextual effect
of negation: clearly “good” and “not very good” indicate opposite sentiment orientations.
> Adapting a
technique of Das and Chen (2001), we added the tag
NOT to every word between a negation word (“not”,
“isn’t”, “didn’t”, etc.) and the first punctuation
mark following the negation word.

To do this we can first get the negation words with a surprisingly effective heuristic:

In [145]:
words = Counter(word for text in X for word in text)

negation_words = [w for (w,c) in words.most_common(10_000) if re.match(".*n[o']t$", w)]
negation_words

['not',
 "doesn't",
 "don't",
 "isn't",
 "can't",
 "didn't",
 "wasn't",
 "aren't",
 "won't",
 "couldn't",
 "wouldn't",
 'cannot',
 "haven't",
 "hasn't",
 "weren't",
 "shouldn't",
 "ain't",
 "hadn't"]

Then we can use a simple finite state machine to add the negation tokens.

In [146]:
punctuation = '!?.,()[];:,"'

def negation_mark(x):
    return 'NOT_' + x

def add_negation_tag(tokens, negation_words=negation_words, punctuation=punctuation, negation_mark=negation_mark):
    in_negation = False
    tagged_tokens = []
    for token in tokens:
        if token in negation_words:
            in_negation = not in_negation
        elif token in punctuation:
            in_negation = False
        elif in_negation:
            token = negation_mark(token)
            
        tagged_tokens.append(token)
        
    return tagged_tokens

def text_add_negation_tag(s: str, **kwargs) -> str:
    return ' '.join(add_negation_tag(tokens=s.split(), **kwargs))

text_add_negation_tag("this isn't a great movie , it is terrible")

"this isn't NOT_a NOT_great NOT_movie , it is terrible"

> For this study, we focused on features based on unigrams (with negation tagging) and bigrams.
> Because training MaxEnt is expensive in the number of features, we limited consideration to (1) the 16165 unigrams appearing at least four times in our 1400-document corpus
> (lower count cutoffs did not yield significantly different results), and (2) the 16165 bigrams occurring most often in the same data (the selected bigrams all occurred at least seven times).
> Note that we did not add negation tags to the bigrams, since we consider bigrams (and n-grams in general) to be an orthogonal way to incorporate context.

There's a slight issue here in using the vocabulary based on the entire dataset; the features should only be selected by the data in each fold otherwise you could be overfitting.

We'll be making lots of this so we'll make a factory function to remove some of the boilerplate.

In [147]:
def make_count_vectorizer(
    max_features=16165,
    input='content',
    token_pattern=r"[^ ]+",
    ngram_range=(1,1),
    preprocessor=None,
    binary=True):
    return CountVectorizer(input=input,
                          token_pattern=token_pattern,
                          ngram_range=ngram_range,
                          preprocessor=preprocessor,
                          max_features=max_features,
                          binary=binary)

unigram_freq_vectorizer = make_count_vectorizer(preprocessor=text_add_negation_tag, binary=False)

unigram_vectorizer = make_count_vectorizer(preprocessor=text_add_negation_tag)

bigram_vectorizer = make_count_vectorizer(ngram_range=(2,2))

## Results

From the README there's an updated Figure 3

| | Features          | # features | NB   | ME   | SVM  |
|-|-------------------|------------|------|------|------|
|1| unigrams (freq.)  | 16162      | 79.0 | n/a  | 73.0 |
|2| unigrams          | 16162      | 81.0 | 80.2 | 82.9 |
|3| unigrams+bigrams  | 32324      | 80.7 | 80.7 | 82.8 |
|4| bigrams           | 16162      | 77.3 | 77.5 | 76.5 |
|5| unigrams+POS      | 16688      | 81.3 | 80.3 | 82.0 |
|6| adjectives        | 2631       | 76.6 | 77.6 | 75.3 |
|7| top 2631 unigrams | 2631       | 80.9 | 81.3 | 81.2 |
|8| unigrams+position | 22407      | 80.8 | 79.8 | 81.8 |


For each feature function (row in the table) we want to apply it followed by each of the 3 types of classifiers

> All results reported below, as well as the baseline results from Section 4,
are the average three-fold cross-validation results on this data.

We can use `cross_val_score` to do the cross-validation.

In [148]:
models = {
    'NB': nb,
    'ME': me,
    'SVM': svm
}

results = {}

def evaluate(vectorizer):
    results = {}
    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('Vectorizer', vectorizer), ('model', model)])
        cv_score = cross_val_score(pipeline, data['text'], y, cv=cv, scoring='accuracy')
        results[model_name] = cv_score.mean()
    return results

### Initial Unigram Results

We can show the results to compare with the table

In [149]:
def display_results(results):
    print('\t'.join(results))
    print('\t'.join([f'{v:0.1%}' for v in results.values()]))

This does a little better than the table from the README, which is surprising especially for Naive Bayes which is deterministic with no tunable parameters.

In [150]:
results['unigram_freq'] = evaluate(unigram_freq_vectorizer)

display_results(results['unigram_freq'])

NB	ME	SVM
80.3%	80.5%	77.0%


### Feature frequency vs. presence

As in the paper using presence features gives a significant lift, but again all the accuracies are much higher here.
In particular Maximum Entropy (Logistic Regression) is higher than Naive Bayes here and much closer to SVM, which suggests there could be poor regularisation in the original.

In [151]:
results['unigram'] = evaluate(unigram_vectorizer)

display_results(results['unigram'])

NB	ME	SVM
81.9%	84.2%	84.4%


### Bigrams 

> Line (3) of the results table shows that bigram
information does not improve performance beyond
that of unigram presence, although adding in the bigrams does not seriously impact the results, even for
Naive Bayes

This is still observed here, but again our results are slightly higher than the authors.

In [152]:
results['unigram_bigram'] = evaluate(FeatureUnion([('unigram', unigram_vectorizer), ('bigram', bigram_vectorizer)]))

display_results(results['unigram_bigram'])

NB	ME	SVM
80.7%	83.8%	84.4%


We have similar observations for bigrams:

> However, comparing line (4) to line (2) shows that relying just on bigrams causes accuracy to decline by as much as 5.8 percentage points. 

In [153]:
results['bigram'] = evaluate(bigram_vectorizer)

display_results(results['bigram'])

NB	ME	SVM
77.6%	78.0%	78.3%


We see a marginally larger decline

In [154]:
{k: f"{results['bigram'][k] - results['unigram'][k] :0.2%}" for k in results['bigram']}

{'NB': '-4.26%', 'ME': '-6.14%', 'SVM': '-6.07%'}

### Parts of speech

> We also experimented with appending POS tags to every word via Oliver Mason’s Qtag program

Unfortunately I can't access [QTag](https://web.archive.org/web/20071005185742/http://www.english.bham.ac.uk/staff/omason/software/qtag.html), but instead will use a much more modern (and likely more accurate) [Averaged Perceptron Tagger](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python) from NLTK.
It's relatively expensive so I'll cache the calculations across models.

In [155]:
import nltk
from functools import lru_cache

nltk.download('averaged_perceptron_tagger')

@lru_cache(maxsize=None)
def pos_tag(doc):
    return nltk.pos_tag(doc.split())

pos_tag('The quick brown fox jumped over the lazy dog.')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eross/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('The', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('jumped', 'VBD'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'JJ'),
 ('dog.', 'NN')]

We can then append the tags as follows:

In [156]:
def pos_marker(doc):
    return ' '.join([token + '_' + tag for token, tag in pos_tag(doc)])

pos_marker(data['text'][0]).split()[:10]

['united_JJ',
 'states_NNS',
 ',_,',
 '1998_CD',
 'u_NN',
 '._.',
 's_NN',
 '._.',
 'release_NN',
 'date_NN']


> However, the effect of this information seems to be a wash: as depicted in
line (5) of Figure 3, the accuracy improves slightly for Naive Bayes but declines for SVMs, and the performance of MaxEnt is unchanged.

We actually get worse results across the board

In [157]:
unigram_pos_vectorizer = make_count_vectorizer(preprocessor=pos_marker)

results['unigram_pos'] = evaluate(unigram_pos_vectorizer)

display_results(results['unigram_pos'])

NB	ME	SVM
80.6%	82.4%	82.5%


In [158]:
display_results(results['unigram'])

NB	ME	SVM
81.9%	84.2%	84.4%


It's not the effect of dropping negations either(in fact that makes Naive Bayes and SVM do slightly better)

In [159]:
unigram_no_negation_vectorizer = make_count_vectorizer(preprocessor=None)


display_results(evaluate(unigram_no_negation_vectorizer))

NB	ME	SVM
82.2%	82.9%	83.3%


> Since adjectives have been a focus of previous work in sentiment detection (Hatzivassiloglou and Wiebe,
2000; Turney, 2002), we looked at the performance of using adjectives alone.

In [160]:
def adjective_extractor(doc):
    adjectives = [token for token, tag in pos_tag(doc) if tag == 'JJ']
    return ' '.join(adjectives)

adjective_extractor(data['text'][0])

"united wide mpaa male full-frontal female graphic sexual frequent theatrical daphne murray liber kimball wild dreary early frontal late early lucrative mtv fast-paced slick flashy mindless wild first eleven convincing much flesh narrative wild real increasingly- improbable predictable serpentine idiotic steven seagal easy unlikely right wrong good next hot young old screen i'll film's erotic impressive wild risqué soft-core generic much lesbian token iron-clad film's full kevin few fully-clothed thirteenth titanic familiar wild john last finely-tuned psychological normal copious real powerful difficult same i mainstream previous wide-release mad box-office quick pretty main wild occasional sam matt florida's blue high curvaceous van theresa daphne skeptical suzie neve similar wild isn't good much ludicrous triple-digit on-screen nice slutty see-through one-piece kevin only interesting right secret wild absurd basic kinetic wild great superficial trash"

>  Yet, the results, shown in line (6) of Figure 3, are relatively poor: the 2633 adjectives provide less useful information than unigram presence.

In [161]:
adjective_vectorizer = make_count_vectorizer(preprocessor=lambda x: adjective_extractor(text_add_negation_tag(x)),
                                             max_features=2633)

results['adjective'] = evaluate(adjective_vectorizer)

display_results(results['adjective'])

NB	ME	SVM
77.2%	76.2%	76.2%


> Indeed, line (7) shows that simply using the 2633 most frequent unigrams is a better choice, yielding performance comparable to that of using (the presence of) all 16165 (line (2)).

In [162]:
unigram_2633_vectorizer = make_count_vectorizer(preprocessor=text_add_negation_tag, max_features=2633)

results['unigram_2633'] = evaluate(unigram_2633_vectorizer)

display_results(results['unigram_2633'])

NB	ME	SVM
81.0%	81.9%	83.2%


### Position

> As a rough approximation to determining this kind of structure, we tagged each word
according to whether it appeared in the first quarter, last quarter, or middle half of the document.

In [163]:
def quartile_marker(doc):
    tokens = doc.split()
    quartile = len(tokens) // 4
    output_tokens = [ token + f'_{i // quartile}' for i, token in enumerate(tokens)]
    return ' '.join(output_tokens)

quartile_marker('this is an example text with several words in it')

'this_0 is_0 an_1 example_1 text_2 with_2 several_3 words_3 in_4 it_4'

I suspect they kept the top vocabulary across the whole document, but we will generate it by quartile.

> The results (line (8)) didn’t differ greatly from using unigrams alone, but more refined notions of position might be more successful.

In [164]:
position_vectorizer =  make_count_vectorizer(preprocessor=quartile_marker, max_features=22430)

results['position'] = evaluate(position_vectorizer)

display_results(results['position'])

NB	ME	SVM
78.7%	79.8%	80.1%


Let's see all our results

In [165]:
import pandas as pd

pd.DataFrame(results).T.style.format('{:0.1%}')

,NB,ME,SVM
unigram_freq,80.3%,80.5%,77.0%
unigram,81.9%,84.2%,84.4%
unigram_bigram,80.7%,83.8%,84.4%
bigram,77.6%,78.0%,78.3%
unigram_pos,80.6%,82.4%,82.5%
adjective,77.2%,76.2%,76.2%
unigram_2633,81.0%,81.9%,83.2%
position,78.7%,79.8%,80.1%


Which is comparable to the original table


| | Features          | # features | NB   | ME   | SVM  |
|-|-------------------|------------|------|------|------|
|1| unigrams (freq.)  | 16162      | 79.0 | n/a  | 73.0 |
|2| unigrams          | 16162      | 81.0 | 80.2 | 82.9 |
|3| unigrams+bigrams  | 32324      | 80.7 | 80.7 | 82.8 |
|4| bigrams           | 16162      | 77.3 | 77.5 | 76.5 |
|5| unigrams+POS      | 16688      | 81.3 | 80.3 | 82.0 |
|6| adjectives        | 2631       | 76.6 | 77.6 | 75.3 |
|7| top 2631 unigrams | 2631       | 80.9 | 81.3 | 81.2 |
|8| unigrams+position | 22407      | 80.8 | 79.8 | 81.8 |


# Discussion

The conclustions from the original still hold up:

> The results produced via machine learning techniques are quite good in comparison to the human-generated baselines discussed in Section 4.
> In terms of relative performance, Naive Bayes tends to do the worst and SVMs tend to do the best, although the differences aren’t very large.
> On the other hand, we were not able to achieve accuracies on the sentiment classification problem comparable to those reported for standard topic-based categorization, despite the several different types of features we tried.
> Unigram presence information turned out to be the most effective; in fact, none of the alternative features we employed provided consistently better performance once unigram presence was incorporated. 

They also give this analysis:

> As it turns out, a common phenomenon in the documents was a kind of “thwarted expectations” narrative, where the author sets up a deliberate contrast to earlier discussion.

We now have much more advanced methods, in particular neural methods that can take the context into account.

## On compute

This paper was relatively easy to reproduce because computers are so much faster than when they wrote it, and software is so much better.

It's worth keeping in mind the change in technology in the last 20 years; the fastest supercomputer in the world at the time, the [Earth Simulator](https://en.wikipedia.org/wiki/Earth_Simulator) could perform just under 36 TFLOPS, about the same as 4 NVIDIA A100's that could be rented today in the cloud for under $5 an hour (AWS was just starting up around this time).

For a more grounded comparison in 2003 Apple released the [Power Mac G5](https://en.wikipedia.org/wiki/Power_Mac_G5) which at the time was a powerful consumer computer, but some [benchmarks from 2017](https://www.phoronix.com/news/PowerMac-Intel-KBL) show it's around 10-100 times slower and a 7th Generation Intel i7. It had 256MB of RAM, where a mid-range laptop today would have at least 8GB, about 30 times more.
This meant recalculating the features 9 times (once per split and per model) was a reasonable thing to do, but would have been very unreasonable at the time.

Also software has come a long way, [Scikit-Learn](https://scikit-learn.org/stable/about.html) started in 2007 and made all of the feature fitting trivial, that at the time would have involved plugging together different systems (likely in C and Java).

It's interesting to think what things may be like in another 20 years.